# Conversational RAG Chatbot with Panel UI

## Overview

This notebook builds an **interactive chatbot UI** using Panel that:
1. Loads the DPDPA PDF into a vector database
2. Provides conversational question answering with **memory** (chat history)
3. Displays sources for transparency and citations

## Key Features

- **Conversational Memory**: Remembers previous questions and answers in the session
- **RAG Pipeline**: Retrieves relevant DPDPA sections before generating answers
- **Panel UI**: Interactive web-based chat interface
- **Source Attribution**: Shows which PDF pages were used for each answer

## Environment Setup

In [ ]:
import os
import openai
import datetime
import panel as pn
from dotenv import load_dotenv, find_dotenv

# Load environment variables
_ = load_dotenv(find_dotenv())
openai.api_key = os.environ['OPENAI_API_KEY']

# Initialize Panel extension for interactive widgets
pn.extension()

# Select appropriate GPT model
current_date = datetime.datetime.now().date()
if current_date < datetime.date(2023, 9, 2):
    llm_name = "gpt-3.5-turbo-0301"
else:
    llm_name = "gpt-3.5-turbo"
print(f"Using model: {llm_name}")

## Import Dependencies

In [ ]:
# Document loading and processing
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Embeddings and Vector Store
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import DocArrayInMemorySearch

# LLM and Chains
from langchain_openai import ChatOpenAI
from langchain_classic.chains import ConversationalRetrievalChain
from langchain_classic.memory import ConversationBufferMemory

# Prompts
from langchain_core.prompts import PromptTemplate

## Load Database Function

This function:
1. Loads a PDF file
2. Splits it into chunks
3. Creates embeddings and stores them in a vector database
4. Returns a ConversationalRetrievalChain with memory

In [ ]:
def load_db(file, chain_type, k):
    """
    Load PDF, create vector database, and return conversational QA chain.
    
    Args:
        file: Path to PDF file
        chain_type: Chain type for retrieval ('stuff', 'map_reduce', 'refine')
        k: Number of documents to retrieve
    
    Returns:
        ConversationalRetrievalChain with memory
    """
    # Load PDF documents
    loader = PyPDFLoader(file)
    documents = loader.load()
    
    # Split documents into chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000, 
        chunk_overlap=150
    )
    docs = text_splitter.split_documents(documents)
    
    # Create embeddings and vector database
    embeddings = OpenAIEmbeddings()
    db = DocArrayInMemorySearch.from_documents(docs, embeddings)
    
    # Create retriever
    retriever = db.as_retriever(
        search_type="similarity", 
        search_kwargs={"k": k}
    )
    
    # Create conversational memory
    memory = ConversationBufferMemory(
        memory_key="chat_history",
        return_messages=True,
        output_key='answer'
    )
    
    # Create LLM
    llm = ChatOpenAI(
        model=llm_name,
        temperature=0
    )
    
    # Create conversational retrieval chain
    qa = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=retriever,
        memory=memory,
        return_source_documents=True,
        chain_type=chain_type
    )
    
    return qa

## Panel Chatbot UI Class

This class creates an interactive chat interface with:
- File upload capability
- Database loading button
- Chat interface with memory
- Source document display

In [35]:
class cbfs(pn.viewable.Viewer):
    """
    Conversational Chatbot with File System (cbfs)
    
    A Panel-based chatbot UI for RAG with conversational memory.
    """
    
    def __init__(self, **params):
        super(cbfs, self).__init__(**params)
        
        # Initialize state
        self.panels = []
        self.loaded_file = "./99-DPDPA.pdf"
        self.qa = None
        
        # Create widgets
        self.file_input = pn.widgets.FileInput(
            accept='.pdf',
            name='Upload PDF'
        )
        
        self.button_load = pn.widgets.Button(
            name="Load Database", 
            button_type='primary'
        )
        self.button_load.on_click(self.load_db_click)
        
        self.button_clear = pn.widgets.Button(
            name="Clear Chat History", 
            button_type='warning'
        )
        self.button_clear.on_click(self.clear_history)
        
        self.inp = pn.widgets.TextInput(
            placeholder='Ask a question about the DPDPA...'
        )
        self.inp.param.watch(self.qa_callback, 'value')
        
        self.bound_button_load = pn.bind(
            self.load_db_click, 
            self.button_load.param.clicks
        )
        
        # Info panel
        self.info_panel = pn.pane.Markdown(
            "**Status:** Click 'Load Database' to start",
            styles={'background-color': '#F6F6F6', 'padding': '10px'}
        )
        
        # Chat display area - dynamic column that updates with chat history
        self.chat_display = pn.Column()
        
    def load_db_click(self, event=None):
        """Load the database when button is clicked"""
        try:
            self.info_panel.object = "**Status:** Loading database..."
            
            # Load the QA chain
            self.qa = load_db(self.loaded_file, "stuff", 4)
            
            self.info_panel.object = f"**Status:** ✅ Database loaded from {self.loaded_file}. Ask questions below!"
            
        except Exception as e:
            self.info_panel.object = f"**Status:** ❌ Error loading database: {str(e)}"
    
    def qa_callback(self, event):
        """Process user question and display answer"""
        question = event.new
        
        if not question or self.qa is None:
            return
        
        try:
            # Get answer from conversational chain
            result = self.qa({"question": question})
            
            # Extract answer and sources
            answer = result['answer']
            source_docs = result.get('source_documents', [])
            
            # Format sources
            sources_text = "\n\n**Sources:**\n"
            if source_docs:
                for i, doc in enumerate(source_docs[:3], 1):
                    page = doc.metadata.get('page', 'N/A')
                    sources_text += f"- Page {page}: {doc.page_content[:150]}...\n"
            else:
                sources_text += "No sources found."
            
            # Create chat message panels
            self.panels.append(
                pn.Row(
                    pn.pane.Markdown(f"**You:** {question}", width=600),
                    styles={'background-color': '#E8F4F8', 'padding': '10px'}
                )
            )
            
            self.panels.append(
                pn.Row(
                    pn.pane.Markdown(f"**Bot:** {answer}{sources_text}", width=600),
                    styles={'background-color': '#F0F0F0', 'padding': '10px'}
                )
            )
            
            # Update the chat display
            self.chat_display.objects = self.panels
            
        except Exception as e:
            self.panels.append(
                pn.Row(
                    pn.pane.Markdown(f"**Error:** {str(e)}", width=600),
                    styles={'background-color': '#FFE6E6', 'padding': '10px'}
                )
            )
            self.chat_display.objects = self.panels
        
        # Clear input
        self.inp.value = ''
    
    def clear_history(self, event=None):
        """Clear chat history and memory"""
        self.panels = []
        self.chat_display.objects = []
        if self.qa:
            self.qa.memory.clear()
        self.info_panel.object = "**Status:** Chat history cleared. Database still loaded."
    
    def __panel__(self):
        """Return the Panel layout"""
        return pn.Column(
            pn.pane.Markdown("# 🤖 DPDPA Chatbot with RAG + Memory"),
            pn.Row(self.button_load, self.button_clear),
            self.info_panel,
            pn.layout.Divider(),
            self.chat_display,
            self.inp,
        )

## Initialize and Display Chatbot

In [36]:
# Create chatbot instance
cb = cbfs()

# Display the chatbot UI
cb

BokehModel(combine_events=True, render_bundle={'docs_json': {'afb2f508-4cfe-4fa8-b42b-1e050b01b57b': {'version…

## Usage Instructions

1. **Load Database**: Click the "Load Database" button to load the DPDPA PDF into memory
2. **Ask Questions**: Type your question in the text input and press Enter
3. **Conversational Context**: The bot remembers previous questions and answers in the session
4. **View Sources**: Each answer includes citations showing which PDF pages were used
5. **Clear History**: Click "Clear Chat History" to reset the conversation (database remains loaded)

## Example Questions

- What are the penalties for a data breach?
- Who is a Data Fiduciary?
- What are the obligations of a Data Fiduciary?
- How should consent be obtained?
- What rights do Data Principals have?

## Test the QA System Directly (Optional)

You can also test the QA chain directly without the UI:

In [38]:
# Load QA chain directly
qa_chain = load_db("./99-DPDPA.pdf", "stuff", 4)

# Ask a question
result = qa_chain({"question": "What are the penalties for a data breach?"})

print("Answer:", result['answer'])
print("\nSources:")
for i, doc in enumerate(result['source_documents'][:2], 1):
    print(f"{i}. Page {doc.metadata.get('page', 'N/A')}: {doc.page_content[:100]}...")

/Users/pratt/technologia/ai-proving-grounds/.venv/lib/python3.12/site-packages/docarray/helper.py:255: SyntaxWarning: invalid escape sequence '\*'
  e.g. '\*.py', '[\*.zip, \*.gz]'


Answer: The penalties for a data breach can extend up to two hundred and fifty crore rupees, as specified in the Schedule mentioned in the Act. The amount of monetary penalty imposed is determined by factors such as the nature, gravity, and duration of the breach, the type of personal data affected, any gains or losses realized by the person, actions taken to mitigate the breach, and the overall impact of the penalty on the person. Additionally, if the Board imposes monetary penalties on a Data Fiduciary in multiple instances, they may advise blocking access to certain information in the interests of the general public.

Sources:
1. Page 13: personal data breach, and to inquire into such personal data breach and impose
penalty as provided i...
2. Page 20: SEC. 1] THE GAZETTE OF INDIA EXTRAORDINARY 21
Breach of provisions of  this Act or rules made thereu...


## Test Conversational Memory

The key feature of this chatbot is **memory** - it remembers previous exchanges:

In [39]:
# First question
result1 = qa_chain({"question": "What is a Data Fiduciary?"})
print("Q1:", result1['answer'])

# Follow-up question (uses memory)
result2 = qa_chain({"question": "What are their main obligations?"})
print("\nQ2:", result2['answer'])

# The bot knows "their" refers to "Data Fiduciary" from the previous question!

Q1: A Data Fiduciary is defined as any person who alone or in conjunction with others determines the purpose and means of processing personal data. They are responsible for complying with data protection regulations and ensuring the security and accuracy of personal data they process.

Q2: The main obligations of a Data Fiduciary include:
1. Complying with the provisions of the data protection Act and rules made thereunder.
2. Being responsible for processing undertaken by itself or on its behalf by a Data Processor.
3. Engaging Data Processors only under a valid contract for offering goods or services to Data Principals.
4. Ensuring completeness, accuracy, and consistency of personal data used in decision-making or disclosed to another Data Fiduciary.
5. Implementing appropriate technical and organizational measures for compliance.
6. Protecting personal data in its possession or control with reasonable security safeguards.
7. Notifying the Board and affected Data Principals in case o